This notebook is designed to be run either locally or in Google Colab. For usage in Google Colab, the following files/folders should be uploaded to the default directory:
- utils.py
- /data/
    

In [1]:
from importlib import reload
import src.utils
reload(src.utils)

ModuleNotFoundError: No module named 'src'

In [4]:
import os
import sys

# Install Prerequisites

In [5]:
if 'google.colab' in str(get_ipython()):
    !pip install datasets
    !pip install transformers
else:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

# Load Data

In [6]:
from src.utils import DataReader

In [7]:
text = DataReader.read_bbc_tech()
print(text[0])
print(len(text))

/Users/jaipancholi/Code/autocomplete_me/src
Mobiles rack up 20 years of useMobile phones in the UK are celebrating their 20th anniversary this weekend.Britain's first mobile phone call was made across the Vodafone network on 1 January 1985 by veteran comedian Ernie Wise. In the 20 years since that day, mobile phones have become an integral part of modern life and now almost 90% of Britons own a handset. Mobiles have become so popular that many people use their handset as their only phone and rarely use a landline.The first ever call over a portable phone was made in 1973 in New York but it took 10 years for the first commercial mobile service to be launched. The UK was not far behind the rest of the world in setting up networks in 1985 that let people make calls while they walked. The first call was made from St Katherine's dock to Vodafone's head office in Newbury which at the time was over a curry house. For the first nine days of 1985 Vodafone was the only firm with a mobile network

# Load Model

In [8]:
from src.utils import TransformerLoader

In [9]:
model = 'gpt2'
tokenizer, model = TransformerLoader.from_huggingface(model, framework='pt')

# Prepare Data

In [10]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset

In [11]:
X_train, X_val = train_test_split(text, test_size=0.1, shuffle=True)

In [12]:
train_dataset = {
    'text': X_train
}
val_dataset = {
    'text': X_val
}
train_dataset = Dataset.from_dict(train_dataset)
val_dataset = Dataset.from_dict(val_dataset)

In [13]:
text_column_name = 'text'
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=[text_column_name],
)

val_tokenized_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=[text_column_name],
)

In [14]:
block_size = 10
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
)

val_lm_datasets = val_tokenized_dataset.map(
    group_texts,
    batched=True,
)

In [15]:
lm_datasets

Dataset(features: {'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 21942)

In [16]:
dataset = {
    'text': text
}
dataset = Dataset.from_dict(dataset)
dataset

Dataset(features: {'text': Value(dtype='string', id=None)}, num_rows: 401)

# Train Model

In [ ]:
from transformers import Trainer
import tensorflow as tf

# Initialize our Trainer
trainer = Trainer(
    model=model,
    # args=training_args,
    train_dataset=lm_datasets,
    eval_dataset=val_lm_datasets,
    tokenizer=tokenizer,
)

trainer.train(
    model_path='./lol'
)

Step,Training Loss


KeyboardInterrupt: 

In [22]:
from transformers import Trainer

from torch import nn
class MyTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        print(logits)
        return 1
#         loss = nn.CrossEntropyLoss()
#         return loss(inputs, logits)
    
trainer = MyTrainer(
    model=model,
    # args=training_args,
    train_dataset=lm_datasets,
    eval_dataset=val_lm_datasets,
    tokenizer=tokenizer,
)

trainer.train(
    model_path='./lol'
)

tensor([[[ -59.9170,  -60.2747,  -65.5996,  ...,  -70.0724,  -67.1970,
           -61.1957],
         [ -77.7487,  -74.3693,  -79.2065,  ...,  -76.3643,  -81.0616,
           -76.5047],
         [-100.5202,  -99.4209, -103.0610,  ..., -106.2792, -102.2401,
          -100.1868],
         ...,
         [-194.1504, -192.0403, -196.1115,  ..., -198.8999, -202.6650,
          -195.4734],
         [ -75.6819,  -74.1729,  -77.9911,  ...,  -79.3521,  -83.0725,
           -75.3379],
         [-108.2669, -108.8594, -112.8886,  ..., -118.8101, -119.5732,
          -108.9292]],

        [[ -62.6072,  -58.7520,  -63.9013,  ...,  -75.4888,  -76.9581,
           -57.6678],
         [ -34.5164,  -35.4127,  -38.8237,  ...,  -41.5125,  -42.0028,
           -34.2283],
         [ -73.2445,  -71.2421,  -76.1751,  ...,  -83.8697,  -79.3562,
           -72.8097],
         ...,
         [-127.2114, -123.3422, -131.2794,  ..., -130.1310, -126.3436,
          -126.1708],
         [ -77.4354,  -75.7528,  -81.135

AttributeError: 'int' object has no attribute 'backward'

# Evaluation

In [ ]:
  import math
  eval_output = trainer.evaluate()

  results = {}
  perplexity = math.exp(eval_output["eval_loss"])
  results["perplexity"] = perplexity

  output_eval_file = os.path.join('.', "eval_results_clm.txt")
  if trainer.is_world_process_zero():
      with open(output_eval_file, "w") as writer:
        
          for key, value in results.items():
              print(f"  {key} = {value}")
              writer.write(f"{key} = {value}\n")

In [ ]:
eval_output

# Generate new sequences

In [ ]:
# user_text = 'Well hello there miss turnip, what a'
# user_text = 'And the stock market has hit'
# user_text = 'not many companies can'
# user_text = 'Then the little leaves'
user_text = 'and the current generation of mobiles using flash technology can'
sequences = tokenizer.encode(user_text, return_tensors="pt")
sequences

In [ ]:
model.to('cuda')
sequences = sequences.to('cuda')

In [ ]:
  
  output_sequences = model.generate(
      # input_ids=sequences['input_ids'],
      input_ids=sequences,
      max_length=50,
      # temperature=args.temperature,
      # top_k=args.k,
      # top_p=args.p,
      # repetition_penalty=args.repetition_penalty,
      do_sample=True,
      # num_return_sequences=0,
  )

In [ ]:
output_sequences

In [ ]:
# Remove the batch dimension when returning multiple sequences
if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

generated_sequences = []

for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    print(text)